# House Pricing Predictions
This is my attempt to solve the data science challenge on [Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview), predicting the house prices given a large number of features. 

## Preliminary work: imports and loading data

In [2]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn

In [3]:
# variables to store the location of the training and test datasets
train_file = "train.csv"
test_file = "test.csv"

In [4]:
df_train_org = pd.read_csv(train_file)
df_test_org = pd.read_csv(test_file)
# df and df_test are copies of the original datasets with target value referred to as y
Y = "y"
df = df_train_org.rename(columns={'SalePrice': Y})
df_test = df_test_org.rename(columns={'SalePrice':Y})

## Data Exploration 
In this section we explore the basic aspects of the provided dataset.

In [5]:
# a function to drop an element to both train and test dataframes
def drop_cols(col_names:list):
    global df, df_test
    try:
        df = df.drop(col_names, axis=1)
        df_test = df_test.drop(col_names, axis=1)
    except:
        print("The column{s} {cols} have already been dropped".
        format(s= "" if (col_names is str or len(col_names) == 1) else "s", cols=str(col_names)))
    

In [6]:
# let's first understand the nature of our data
print(df.shape) 
# each sample is described by 81 features. This number if relatively high.

print((df['Id'].values == range(1 ,len(df) + 1)).all()) 
# as we can see the Id column is merely for ennumeriation purposes. It can be either dropped or set as an index.
drop_cols("Id")


(1460, 81)
True


### Categorical and Numerical features
Let's consider the different types of features. First, we divide them into numerical and non-numerical. The non-numerical are definitely categorical (or can be made as such). As for numerical, columns with int values, can be considered categorical if the number of unique values is limited.

In [7]:
# consider non-numerical values
object_type = "object"
cat_type = 'category'
non_num_cols = df.select_dtypes([object_type, cat_type]).columns
print(non_num_cols)
num_cols = df.select_dtypes(np.number).columns
print(num_cols)

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'Kitch

In [8]:
# let's consider the subset of numerical columns with few discrete values
num_discrete = df.select_dtypes('int64').columns
num_dis_count = [len(df[num_d].unique()) for num_d in num_discrete]  
print(dict(zip(num_discrete, num_dis_count)))

# we can see that a "MSSubClass" is a categorical feature
# a number of features are not categorical by say, but can be treated as such: Bath related features, Fireplaces, GarageCars, and most importantly
# OverallQual and OveralCond


{'MSSubClass': 15, 'LotArea': 1073, 'OverallQual': 10, 'OverallCond': 9, 'YearBuilt': 112, 'YearRemodAdd': 61, 'BsmtFinSF1': 637, 'BsmtFinSF2': 144, 'BsmtUnfSF': 780, 'TotalBsmtSF': 721, '1stFlrSF': 753, '2ndFlrSF': 417, 'LowQualFinSF': 24, 'GrLivArea': 861, 'BsmtFullBath': 4, 'BsmtHalfBath': 3, 'FullBath': 4, 'HalfBath': 3, 'BedroomAbvGr': 8, 'KitchenAbvGr': 4, 'TotRmsAbvGrd': 12, 'Fireplaces': 4, 'GarageCars': 5, 'GarageArea': 441, 'WoodDeckSF': 274, 'OpenPorchSF': 202, 'EnclosedPorch': 120, '3SsnPorch': 20, 'ScreenPorch': 76, 'PoolArea': 8, 'MiscVal': 21, 'MoSold': 12, 'YrSold': 5, 'y': 663}


## Data Cleaning
Certain columns might contain corrupted data and thus require cleaning. I will start with categorical columns. 


### Cleaning categorical columns 
The main procedure is as follows:
* replace the values that do not belong to the data description set of values by the one described there, mainly typos in string typed values
* in the worst case drop rows that have values significantly different from the pre-determined categories


In [9]:
for col, uni_values in zip(non_num_cols, [df[col].unique() for col in non_num_cols]):
    print(col)
    print(uni_values)
    print()

MSZoning
['RL' 'RM' 'C (all)' 'FV' 'RH']

Street
['Pave' 'Grvl']

Alley
[nan 'Grvl' 'Pave']

LotShape
['Reg' 'IR1' 'IR2' 'IR3']

LandContour
['Lvl' 'Bnk' 'Low' 'HLS']

Utilities
['AllPub' 'NoSeWa']

LotConfig
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']

LandSlope
['Gtl' 'Mod' 'Sev']

Neighborhood
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']

Condition1
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']

Condition2
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']

BldgType
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']

HouseStyle
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']

RoofStyle
['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']

RoofMatl
['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']

Exterior1st
['Vin

In [10]:
# investigating the output of the previous cell as well as inspecting the content of the documentation
# suggests a couple of corrupted values in certain columns such as Exterior2nd

# let's define a function to replace these values both in the training and test dataframes

def replace_values(col_names:list , wrong_correct:list):
    assert (isinstance(col_names, str) and isinstance(wrong_correct, dict)) or all([isinstance(l, dict) for l in wrong_correct]) and isinstance(col_names, list) 
    global df, df_test
    try:
        for col, dic in zip(col_names, wrong_correct):
            for k, v in dic.items():
                df[col] = df[col].replace(k, v)
    except:
        print("Something is wrong check again !!")


In [11]:
# we have the value C (all) is correputed in MSZoning
mszoning = "MSZoning"
correct_mszoning = {"C (all)": "C"}
ext2 = "Exterior2nd"
correct_ext2 = {"Brk Cmn": "BrkComm", "CmentBd": "CemntBd"}

ms_ext = [mszoning, ext2]
correct = [correct_mszoning, correct_ext2]

replace_values(ms_ext, correct)


### Cleaning numerical columns
This task is slightly trickier as it might require domain expertise. The main procedure is as follows:
1. replace (or drop) values that contradict general common sense, for instance negative areas, months cannot be more than $12$
2. consider the relationship between certain rows. values in a certain columns cannot be smaller / larger than the corresponding values in other columns. This step require more careful study of the nature of the problem.

In [12]:
# let's first display the numerical columns
print(num_cols)

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'y'],
      dtype='object')


In [13]:
# let's consider areas
areas = [col for col in num_cols if ("area" in col.strip().lower())]
# inverstiagating the data description, the term SF generally refers to surface (area)
areas.extend([col for col in num_cols if "SF" in col.strip()]) 
print(areas)
# verify all values are positive
areas_with_neg = [any(df[area] < 0) for area in areas]
areas_with_neg = [area for area, a in zip(areas, areas_with_neg) if a]
print(areas_with_neg)
# as we can see all areas-values are positive

['LotArea', 'MasVnrArea', 'GrLivArea', 'GarageArea', 'PoolArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'WoodDeckSF', 'OpenPorchSF']
[]


The data description indicats that 
* OverallQuall and OverallCond should belong to the interval [1, 10]
* YearRemodAdd is year of remodel, thus it should be larger or equal to YearBuilt
* The relationship between the the year where the garage was built and the other year features should be investigated.
* MSSubClass represents a label encoding of the different types of houses: values should belong to predetermined set of values specified in the data description
* MoSold is the month where the house was sold. it should belong to [1, 12]
* it might be worthwhile investigatinv any abnormalities in the relationship between "BsmtFinSF1", "BsmtUnfSF", "BsmtFinSF2" and "TotalBsmtSF" 
 

In [14]:
# first of all let's define a method to replace the names of features in both training and test datasets

def new_col_names(old_new_names:dict):
    global df, df_test
    try:
        df = df.rename(columns=old_new_names)
        df_test = df_test.rename(columns=old_new_names)
    except:
        df_no_col = [col for col in old_new_names.keys if col not in df.columns]
        df_test_no_col = [col for col in old_new_names.keys if col not in df_test.columns]
        print("{cols} are not in the {dataf}".format(df_no_col, "training dataset"))
        print("{cols} are not in the {dataf}".format(df_test_no_col, "test dataset"))


In [15]:
old_new_cols = {"OverallQual": "qua", 
"OverallCond": "cond", "YearBuilt": "Yb", "YearRemodAdd": "Yr", "MSSubClass": "mss" , 
"BsmtFinSF1": "bSF1", "BsmtFinSF2": "bSF2", "BsmtUnfSF": "bubf", "GarageYrBlt":"GYb"}
new_col_names(old_new_cols)

In [16]:
# verify the integrity of overall quality and condition features
qua = "qua"
cond = "cond"
print(all(df[qua].isin(range(1, 11))))
print(all(df[cond].isin(range(1, 11))))

# verify the remodeling and building years features
yb = "Yb"
yr = "Yr"
print(df[df[yb] > df[yr]].empty) # the resulting dataframe is empty: no problems with neither of these features

# verify the MSubclass features
mss = "mss"
mss_values = [20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 150, 160, 180, 190]

print(df[~df[mss].isin(mss_values)].empty) # all values from "MSsubclass" feature are under check

# verify the month feature

print(df[~df["MoSold"].isin(range(1, 13))].empty) # all values of the month feature are correct

True
True
True
True
True


In [17]:
# let' define a function that applies a function to either 
# the whole dataframe or certain columns on the dataframe

def apply_functions(funcs, col_names=None):
    # either have one function passed that should be applied to the whole dataframe
    # or have an equal number of columns and functions where each funtion will be applied to the corresponding column
    
    all_data = callable(funcs) and col_names is None
    col_funcs = True
    #  if the funcs argument is indeed a function, then the code below will raise an error 
    try:
        col_funcs = (all([callable(f) for f in funcs]) and len(funcs) == len(col_names))
    except:
        col_funcs = False
    
    assert all_data or col_funcs
    
    global df, df_test
    if col_names is None: # if the function is to be applied to the whole dataframe
        df = df.apply(funcs, axis=1)
        df_test = df_test.apply(funcs, axis=1)
    else:
        for col, f in zip(col_names, funcs):
            df[col] = df[col].apply(f)
            df_test[col] = df_test[col].apply(f)
    

In [18]:
# let's consider the year where the garage was built
gyb = "GYb"
print(df[df[gyb] < df[yb]][[gyb, yb, yr]])

print(df_test[df_test[gyb] < df_test[yb]][[gyb, yb, yr]]) 
# with few exceptions the year where the garage if it is before the year where the house was built is generally few years earilier
# which suggests that the garage was meant to be built with the house, yet the house took slightly longer to complete.

# if GYb is less than Yb then we will set to Yb.
def set_garage_year(row):
    if row[gyb] < row[yb]:
        row[gyb] = row[yb]
    return row
# set the changes in both train and test data
print(callable(set_garage_year))
apply_functions(set_garage_year)

print(df[df[gyb] < df[yb]][[gyb, yb, yr]]) 
print(df_test[df_test[gyb] < df_test[yb]][[gyb, yb, yr]])
# the changes were applied to both data sets

         GYb    Yb    Yr
29    1920.0  1927  1950
93    1900.0  1910  1998
324   1961.0  1967  2007
600   2003.0  2005  2005
736   1949.0  1950  1950
1103  1954.0  1959  1959
1376  1925.0  1930  1950
1414  1922.0  1923  2000
1418  1962.0  1963  1963
         GYb    Yb    Yr
61    1956.0  1959  1959
116   2009.0  2010  2010
345   1920.0  1935  1998
380   1960.0  1978  1978
435   1940.0  1941  1950
437   1926.0  1935  1950
662   1925.0  1945  1995
803   2005.0  2006  2007
1049  2005.0  2006  2006
True
Empty DataFrame
Columns: [GYb, Yb, Yr]
Index: []
Empty DataFrame
Columns: [GYb, Yb, Yr]
Index: []


## Imputing missing values
The second step is imputing the missing values and making sure no Nan values are passed to our machine learning models. The main procedure is as follows:
1. drop all columns with a nan ratio exceeding a certain treshhold
2. imput the missing values. The strategy depends mainly on the feature in question:
    * if the feature is highly related to other features, then a highly accurate and natural value can be deduced
    * some statistical value could be used to impute the missing values
    * investigating related features could help come up with an aggregated value when the data is grouped by a number of features.

### 1st strategy: drop columns

In [19]:
# let's first discover which columns have Nan values
nan_values = df.isna().sum()
cols_nan = nan_values[nan_values > 0]
print(cols_nan)
NAN_THRESHOLD = 0.8 # all columns with more than 0.8 nan values will be dropped
cols_nan = cols_nan / len(df)
# print(cols_nan)
cols_nan_drop = cols_nan[cols_nan > NAN_THRESHOLD].index
print(cols_nan_drop) 

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GYb               81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64
Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


As we can see: Alley, PoolQC, fence and MiscFeature have an extremely high ratio of nan values. 
before dropping these columns, it is necessary to drop features tightly related to those.  
Investigating the data description reveals that:  
* PoolQC is related to PoolArea
* MiscFeature is related to MiscVal


In [20]:
# let's better understand those relations
pq = "PoolQC"
pa = "PoolArea"
print(df[df[pq].isna()][pa].value_counts()) 
# Nan values are associated with 0 pool area, which means that there is no pool in the first place
# an extremely high ratio of houses do not have a pool.
# let's consider the prices of houses with pool
print(df[~df[pq].isna()][Y]) # we can see that the values too few and quite variant as well
# thus is it better to drop this column

0    1453
Name: PoolArea, dtype: int64
197     235000
810     181000
1170    171000
1182    745000
1298    160000
1386    250000
1423    274970
Name: y, dtype: int64


In [21]:
misf = "MiscFeature"
misv = "MiscVal"
print(df[df[misf].isna()][misv].value_counts()) # we can see that houses with Nan on MiscFeature 
# have 0 on MiscVal which means that they do not any additional features to mention
# both of these features should be dropped

0    1406
Name: MiscVal, dtype: int64


In [22]:
# let's drop the features in questions
cols_nan_drop = cols_nan_drop.values.tolist() + [misv, pa]
drop_cols(cols_nan_drop)

### 2nd strategy: deduce values

In [23]:
# let's consider the columns left with nan values
nan_values = df.isna().sum()
cols_nan = nan_values[nan_values > 0]
print(cols_nan)

LotFrontage     259
MasVnrType        8
MasVnrArea        8
BsmtQual         37
BsmtCond         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GYb              81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64


The values with Nan can be divided into categories:
* Masonry veneer 
* Basement
* Garage
* FirePlace
* LotFrontage (only one column) and Electrical


In [24]:
def set_nan(col_names:list, fill_values:list):
    one = isinstance(col_names, str) and isinstance(fill_values, str)
    try:
        many = len(col_names) == len(fill_values)
    except:
        many = False
    assert one or many 
    global df, df_test
    if many: 
        for col, v in zip(col_names, fill_values):
            df[col] = df[col].fillna(v)
            df_test[col] = df_test[col].fillna(v)
    else:
        df[col_names] = df[col_names].fillna(fill_values)
        df_test[col_names] = df_test[col_names].fillna(fill_values)

In [25]:
# let's consider Masonry Veneer columns
msvt = "MasVnrType"
msva = "MasVnrArea"

print(df[(df[msvt].isna()) | (df[msva].isna())][[msva, msvt]]) 
# we can see that type and are either both nan or both non-nan
# a reasonable assumption is that there is no Masonry Veneer

set_nan([msvt, msva], ['None', 0])


      MasVnrArea MasVnrType
234          NaN        NaN
529          NaN        NaN
650          NaN        NaN
936          NaN        NaN
973          NaN        NaN
977          NaN        NaN
1243         NaN        NaN
1278         NaN        NaN


In [26]:
# let's consider the garage columns
gt = "GarageType"
gf = "GarageFinish"
gc = "GarageCars"
ga = "GarageArea"
gcond = "GarageCond"
gqua = "GarageQual"

g = [gt, gyb, gf, gc, ga, gcond, gqua]
print(df[df[gt].isna() | df[gyb].isna() | df[gf].isna() | df[gcond].isna() | df[gqua].isna()][ga].value_counts())
# if any of the garage features in a certain row is set to Nan, then its area is 0
# which means there is no garage

# let's first consider the Garage built year feature for such rows
print(df[df[gt].isna() | df[gf].isna() | df[gcond].isna() | df[gqua].isna()][gyb].value_counts())

set_nan(g, ["NA", 0, "NA", 0, 0, "NA", "NA"])

garage_old_new = {gt: "gt", gf:"gf", gc:"gc", ga:"ga", gcond:"gcond", gqua:"gqua"}

new_col_names(garage_old_new)

gt = "gt"
gf = "gf"
gc = "gc"
ga = "ga"
gcond = "gcond"
gqua = "gqua"


0    81
Name: GarageArea, dtype: int64
Series([], Name: GYb, dtype: int64)


In [27]:
# let's consider the basement nan-values
bqua = "BsmtQual"
bcond = "BsmtCond"
bexp = "BsmtExposure"
bf2 = "BsmtFinType2"
bf1 = "BsmtFinType1"
baf1 = "bSF1"
baf2 = "bSF2"

# BsmtQual         37
# BsmtCond         37
# BsmtExposure     38
# BsmtFinType1     37
# BsmtFinType2     38

print(df[df[bqua].isna() & df[bcond].isna() & df[bexp].isna() & df[bf1].isna() ][baf1].value_counts())
# we can assume that Nan values for each of these basement features reflect No basement

set_nan([bqua, bcond, bexp, bf1, bf2], ["NA"] * 5)
# change the basement's features 

basement_new_old = {bqua: "bqua", bcond: "bcond", bexp: "bexp", bf1:"bf1", bf2: "bf2"}
new_col_names(basement_new_old)

bqua = "bqua"
bcond = "bcond"
bexp = "bexp"
bf2 = "bf2"
bf1 = "bf1"
baf1 = "baf1"
baf2 = "baf2"


0    37
Name: bSF1, dtype: int64


In [28]:
# let's check the rest real quick
# LotFrontage     259
# Electrical        1
# FireplaceQu     690

firequa = "FireplaceQu"
firep = "Fireplaces"
print(df[df[firequa].isna()][firep].value_counts())
# so nan values refer to NOn existing fire places
set_nan(firequa, "NA")

e = "Electrical"
print(df[e].value_counts())
lf = "LotFrontage"
# we can assume that the only missing value is SBrkr with a high statistical possibility
# the simplest solution for LotFrontage is to nan values to 0
set_nan([e, lf], ["SBrkr", 0])

fire_new_old = {firequa:"firequa", firep:"firep"}
new_col_names(fire_new_old)

firequa = "firequa"
firep = "firep"


0    690
Name: Fireplaces, dtype: int64
SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64


In [29]:
print(df[df.isna()].sum().sum())

0.0


## Encode the categorial features

In [30]:
# first let's consider once again categorical and numerical features
non_num_cols = df.select_dtypes([object_type, cat_type]).columns
# print(non_num_cols)
num_cols = df.select_dtypes(np.number).columns
# print(num_cols)

In [31]:
# store the current state of the dataframes of later modifications
df_cat = df.copy()
df_t_cat = df_test.copy()

In [32]:
# ordinal_non_num = ["LotShape", "Utilities", "LandSlope", "ExterQual"
# , "ExterCond", "bqua", "bcond", "bexp", "bf1", "bf2", "Heating", "HeatingQC", "Electrical", 
# "KitchenQual", "Functional", "firequa", "gf", "gqua", "gcond", "PavedDrive"]
 
# let's define the orders for each of the ordinal columns
usual_levels = ["NA", "Po", "Fa", "TA", "Gd", "Ex"]

ordered_levels = {
    "ExterQual": usual_levels,
    "ExterCond": usual_levels,
    "bqua": usual_levels,
    "bcond": usual_levels,
    "HeatingQC": usual_levels,
    "KitchenQual": usual_levels,
    "firequa": usual_levels,
    "gqua": usual_levels,
    "gcond": usual_levels,
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "bexp": ["NA", "No", "Mn", "Av", "Gd"],
    "bf1": ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "bf2": ["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "Functional": ["Sal", "Sev", "Maj1", "Maj2", "Mod", "Min2", "Min1", "Typ"],
    "gf": ["NA", "Unf", "RFn", "Fin"],
    "PavedDrive": ["N", "P", "Y"],
    "Utilities": ["NoSeWa", "NoSewr", "AllPub"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"],
}

In [33]:
# let's encode ordinal data
def cat_to_ord(col:list, categories:list, ordered:bool=True):
    global df, df_test
    final_cat = categories if ordered else categories[::-1]
    df[col] = df[col].apply(dict(zip(final_cat, range(0, len(final_cat)))).get)
    df_test[col] = df_test[col].apply(dict(zip(final_cat, range(0, len(final_cat)))).get)


for k, v in ordered_levels.items():
    cat_to_ord(k, v)

In [34]:
for k in ordered_levels.keys():
    print(df[k].value_counts())

3    906
4    488
5     52
2     14
Name: ExterQual, dtype: int64
3    1282
4     146
2      28
5       3
1       1
Name: ExterCond, dtype: int64
3    649
4    618
5    121
0     37
2     35
Name: bqua, dtype: int64
3    1311
4      65
2      45
0      37
1       2
Name: bcond, dtype: int64
5    741
3    428
4    241
2     49
1      1
Name: HeatingQC, dtype: int64
3    735
4    586
5    100
2     39
Name: KitchenQual, dtype: int64
0    690
4    380
3    313
2     33
5     24
1     20
Name: firequa, dtype: int64
3    1311
0      81
2      48
4      14
5       3
1       3
Name: gqua, dtype: int64
3    1326
0      81
2      35
4       9
1       7
5       2
Name: gcond, dtype: int64
0    925
1    484
2     41
3     10
Name: LotShape, dtype: int64
2    1382
1      65
0      13
Name: LandSlope, dtype: int64
1    953
3    221
4    134
2    114
0     38
Name: bexp, dtype: int64
1    430
6    418
5    220
4    148
3    133
2     74
0     37
Name: bf1, dtype: int64
1    1256
3      54
2      46


In [35]:
# let's encode non ordinal data using the label encoder
from sklearn.preprocessing import LabelEncoder
non_ord_cat = [cat for cat in non_num_cols if cat not in ordered_levels.keys()]

le = LabelEncoder()

for col in non_ord_cat:
    df[col] = le.fit_transform(df[col].values)    

## FeatureEngineering
In this section we will work on improving the performance by creating a number of sythetic features out of the given ones. 

In [36]:
# let's save the dataframe before introducing any new feature
df_base = df.copy()
df_test_base = df_test.copy()

### Baseline performance
Creating a baseline model with the initial (left) features gives us a ground on which I base my next decisions. I will use the same performance metric as in the competition. The baseline model would be a sophisticated RandomForest model.

In [37]:
# definining the cross validation procedure
from sklearn.model_selection import KFold
n_splits = 5 
random_state = 3
shuffle = True
kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

In [38]:
print(sklearn.metrics.get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'top_k_accuracy', 

In [39]:
# the main score of the competition is the square of log error squared
# let's define a function to calculate a model's performance according to this metric
scoring = "neg_mean_squared_error"
from sklearn.model_selection import cross_val_score
def model_performance(X, y, model):
    global kf, scoring
    log_y = np.log(y)
    score = cross_val_score(model, X, log_y, cv=5, scoring=scoring)
    return np.sqrt(-score.mean()) 

In [40]:
X = df.copy()
y = X.pop(Y)

from sklearn.ensemble import RandomForestRegressor

rf_base = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=3)
base_score = model_performance(X, y, rf_base)
print(base_score)

0.15927576428363702


### Informative features: mutual information
The mutual information is quite a powerful and general technique to determine the relevance of features with respect to the target variable.


In [41]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    X = X.copy()
    # discrete features are the ones with type int
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]

    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [42]:
mi_scores = make_mi_scores(X, y)
mi_scores

qua             5.786503e-01
Neighborhood    5.253900e-01
ga              4.927946e-01
GrLivArea       4.328080e-01
Yb              4.087151e-01
                    ...     
Condition2      2.664787e-03
3SsnPorch       7.615664e-04
Street          3.666896e-04
Utilities       1.887379e-15
MoSold          0.000000e+00
Name: MI Scores, Length: 73, dtype: float64

In [43]:
relevant_feats = mi_scores[mi_scores > 0.01].index.values.tolist() + [Y]
df = df.loc[:, relevant_feats]

In [44]:
X = df.copy()
y = X.pop(Y) 

new_score = model_performance(X, y, rf_base)
print(new_score) # a very small gain is achieved out of removing the unformative features

0.15887773882723472


In [45]:
print(mi_scores.head(20))

qua             0.578650
Neighborhood    0.525390
ga              0.492795
GrLivArea       0.432808
Yb              0.408715
TotalBsmtSF     0.399518
LotArea         0.392427
gc              0.360102
bqua            0.331398
ExterQual       0.325150
KitchenQual     0.322472
1stFlrSF        0.283265
mss             0.278183
Yr              0.275659
FullBath        0.273156
gf              0.262945
GYb             0.259218
LotFrontage     0.216092
firequa         0.209348
TotRmsAbvGrd    0.207962
Name: MI Scores, dtype: float64


In [ ]:
# 

In [ ]:
# let's consider the overall quality of a house
# as well as the condition
x = df.index.values.tolist()
plt.scatter(x, df[col].values, color='blue')
plt.scatter(x, df[qua].values, color='red')
plt.title("{col} and quality".format(col=col))
plt.show()


In [ ]:
print(df.plot(kind='scatter', y=Y, x=cond))

In [48]:
# the quality feature on its own is quite informativa however, it is somehow related to other features 
corr_with_qua = df.corr()[qua]
corr_with_qua = corr_with_qua[(corr_with_qua > 0.4) | (corr_with_qua < -0.4)].sort_values()

x = df.index.values.tolist()
qua_values = df[qua].values

# for col in corr_with_qua.index:
#     plt.scatter(x, df[col].values, color='blue')
#     plt.scatter(x, df[qua].values, color='red')
#     plt.title("{col} and quality".format(col=col))
#     plt.show()
